In [ ]:
import ee
import folium
import geemap
import geopandas as gpd
import altair as alt

import sys
sys.path.insert(0, "../../src")

from etl import boundsBuffer, sizeCode, mosaicByDate
from viz import *

# Fire Selection

In [ ]:
# Using EPSG:4326 for plotting
crs = 'EPSG:4326'

# CA counties
counties = gpd.read_file("../../data/CA_Counties/CA_Counties_TIGER2016.shp")
counties.to_crs(crs, inplace=True)

sfLowerBound = counties[counties["NAME"]=="San Francisco"]["geometry"].bounds["maxy"].values[0]

norCal = counties.bounds.apply(lambda x: x[3]>sfLowerBound, axis=1)
norCalCounties = counties[norCal]

# CA wildfires 2013-2020
firesDf = gpd.read_file("../../data/California_Fire_Perimeters_(all).geojson")
firesDf.to_crs(crs, inplace=True)

firesDf = firesDf[(firesDf["GIS_ACRES"] >= 10000) & (~firesDf["CONT_DATE"].isna()) & (firesDf["YEAR_"].astype(int)>=2013)]
firesDf["GIS_ACRES"] = firesDf["GIS_ACRES"].round(2)

norCalFires = firesDf.bounds.apply(lambda x: x[1]>sfLowerBound, axis=1)
firesDf = firesDf[norCalFires].reset_index(drop=True)

firesDf.shape

In [ ]:
# All fires > 10000 acres from 2013-2020 in Northern California
alt.layer(alt.Chart(norCalCounties
            ).mark_geoshape(fill="#E6E6E6", stroke="black"
            ).encode(tooltip=["NAME"]),
          alt.Chart(firesDf[~firesDf["FIRE_NAME"].isin(["AUGUST COMPLEX FIRES", "NORTH COMPLEX"])]
            ).mark_geoshape(stroke="black"
            ).encode(color="GIS_ACRES:Q",
                     tooltip=["FIRE_NAME", "ALARM_DATE", "CONT_DATE", "REPORT_AC", "GIS_ACRES"])
   ).properties(width=700, height=700)

In [ ]:
fireSubset = firesDf["FIRE_NAME"].apply(lambda x: x in ["ATLAS","BALD","BUCK","BUTTE","CALDWELL","CAMP","CARR ",
                                                        "CASCADE", "COVE","FRYING PAN","HAPPY","KINCADE",
                                                        "KING","OAK","REDWOOD VALLEY","ROCKY  ","WALKER"])

subsetDf = firesDf[fireSubset].sort_values(by="FIRE_NAME")
subsetDf["sizeCode"] = subsetDf["GIS_ACRES"].apply(sizeCode)


# bounding box as its own df/layer
bboxDf = gpd.GeoDataFrame(subsetDf.loc[:, ~(subsetDf.columns=="geometry")],
                          geometry=subsetDf["geometry"].apply(lambda x: boundsBuffer(x.bounds).difference(x)))

boundsDf = gpd.GeoDataFrame(subsetDf.loc[:, ~(subsetDf.columns=="geometry")],
                            geometry=subsetDf["geometry"].apply(lambda x: boundsBuffer(x.bounds)))

In [ ]:
# Selected fires with bounding box + 7.5% buffer
alt.layer(alt.Chart(norCalCounties
            ).mark_geoshape(stroke="black", fill="#E6E6E6"
            ).encode(tooltip=["NAME"]),
          alt.Chart(bboxDf).mark_geoshape(stroke="red", fill="pink"),
          alt.Chart(subsetDf
            ).mark_geoshape(stroke="red", fill="lightgreen"
            ).encode(#color="GIS_ACRES:Q",
                     tooltip=["FIRE_NAME", "ALARM_DATE", "CONT_DATE",
                              "REPORT_AC", "GIS_ACRES", "sizeCode"]),
   ).properties(width=700, height=700)

# Image selection

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# pre-fire image selection
for fireName, start, geometry in boundsDf[["FIRE_NAME", "ALARM_DATE", "geometry"]].values:
    geometry = ee.Geometry.Rectangle(list(geometry.bounds))
    center = geometry.centroid().getInfo()["coordinates"][::-1]

    l8_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
             ).filterBounds(geometry
             ).filterDate(ee.Date(start).advance(-48, "day"), start
             ).sort("system:time_start")
             
    l8_sr = mosaicByDate(l8_sr)

    m = folium.Map(location=center, zoom_start=11.25)

    for i in range(l8_sr.size().getInfo()):
        image = ee.Image(l8_sr.get(i)).clip(geometry)
        m.add_ee_layer(image,
                       {"bands": ["SR_B7", "SR_B5", "SR_B3"], 
                        "gamma": [1.1, 1.1, 1],
                        "min": 1000, "max": 25000},
                        image.get("mosaicDate").getInfo())
        
    m.add_child(folium.LayerControl())
    print(fireName, start)
    display(m)
    print("\n \n")

In [ ]:
# post-fire image selection
for fireName, start, geometry in boundsDf[["FIRE_NAME", "CONT_DATE", "geometry"]].values:  
    geometry = ee.Geometry.Rectangle(list(geometry.bounds))
    center = geometry.centroid().getInfo()["coordinates"][::-1]

    l8_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
             ).filterBounds(geometry
             ).filterDate(ee.Date(start).advance(-16, "day"), ee.Date(start).advance(60, "day")
             ).sort("system:time_start")
    
    l8_sr = mosaicByDate(l8_sr)
    
    m = folium.Map(location=center, zoom_start=11.25)

    for i in range(l8_sr.size().getInfo()):
        image = ee.Image(l8_sr.get(i)).clip(geometry)
        m.add_ee_layer(image,
                       {"bands": ["SR_B7", "SR_B5", "SR_B3"], 
                        "gamma": [1.1, 1.1, 1],
                        "min": 1000, "max": 25000},
                        image.get("mosaicDate").getInfo())
        
    m.add_child(folium.LayerControl())
    print(fireName, start)
    display(m)
    print("\n \n")

In [ ]:
# Selected pre-post fire image dates
boundsDf["pre-date"] = ["2017-10-04", "2014-07-24", "2017-08-24", "2015-09-06", "2020-07-15", "2018-10-07",
                        "2018-07-10", "2017-10-04", "2017-07-16", "2014-07-15", "2015-07-25", "2019-10-01",
                        "2014-09-03", "2017-07-14", "2017-09-25", "2015-07-27", "2019-09-01"]


boundsDf["post-date"] = ["2017-11-05", "2014-09-10", "2017-12-14", "2015-10-24", "2020-09-01", "2018-12-26",
                         "2018-10-14", "2017-11-05", "2017-10-04", "2014-10-03", "2015-09-20", "2019-11-18",
                         "2014-10-21", "2017-10-18", "2017-10-27", "2015-08-19", "2020-02-24"]

In [ ]:
# selected pre-post fire images
for fireName, preFireDate, postFireDate, geometry in boundsDf[["FIRE_NAME", 'pre-date', "post-date", "geometry"]].values:
    geometry = ee.Geometry.Rectangle(list(geometry.bounds))
    center = geometry.centroid().getInfo()["coordinates"][::-1]

    preFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2" 
                                ).filterBounds(geometry
                                ).filterDate(preFireDate,
                                                ee.Date(preFireDate).advance(1, "day")))

    postFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
                                ).filterBounds(geometry
                                ).filterDate(postFireDate,
                                                ee.Date(postFireDate).advance(1, "day")))
    
    preFireImage_l8, postFireImage_l8 = ee.Image(preFireImage_l8.get(0)), ee.Image(postFireImage_l8.get(0))

    m = folium.Map(location=center, zoom_start=11.25)
    m.add_ee_layer(preFireImage_l8.clip(geometry), 
                   {"bands": ["SR_B7", "SR_B5", "SR_B3"], 
                    "gamma": [1.1, 1.1, 1],
                    "min": 1000, "max": 25000},
                     preFireImage_l8.get("mosaicDate").getInfo())
    
    m.add_ee_layer(postFireImage_l8.clip(geometry), 
                   {"bands": ["SR_B7", "SR_B5", "SR_B3"], 
                    "gamma": [1.1, 1.1, 1],
                    "min": 1000, "max": 25000},
                     postFireImage_l8.get("mosaicDate").getInfo())
    
    m.add_child(folium.LayerControl())     
    print(fireName)       
    display(m)
    print("\n \n")

In [ ]:
subsetDf.to_file("../../data/burned/fireBounds.shp")  # fire geometry
bboxDf.to_file("../../data/unburned/bbox.shp")   # differenced bounding box with buffer
boundsDf.to_file("../../data/bounds/bounds.shp")   # bounding box with buffer